# Proyecto X

## Lectura de los datos y exploración de los encabezados de los archivos.

In [1]:
import pandas as pd

In [2]:
geolocation = pd.read_csv('./content/Copia de ecommerce_geolocation_dataset.csv')
customers = pd.read_csv('./content/Copia de ecommerce_customers_dataset.csv')
order_items = pd.read_csv('./content/Copia de ecommerce_order_items_dataset.csv')
order_payments = pd.read_csv('./content/Copia de ecommerce_order_payments_dataset.csv')
order_reviews = pd.read_csv('./content/Copia de ecommerce_order_reviews_dataset.csv')
orders = pd.read_csv('./content/Copia de ecommerce_orders_dataset.csv')
products = pd.read_csv('./content/Copia de ecommerce_products_dataset.csv')
sellers = pd.read_csv('./content/Copia de ecommerce_sellers_dataset.csv')
category_names = pd.read_csv('./content/Copia de product_category_name_translation.csv')

## BBT1-14 Creación de la base de datos y carga de datos

**Descripción**
Crear una conexión a la base de datos de SQLite llamada ecommerce.db
Importar los archivos y Guardar los DataFrames en la base de datos como tablas:

ecommerce_customers_dataset.csv

ecommerce_order_items_dataset.csv

ecommerce_order_payments_dataset.csv

ecommerce_order_reviews_dataset.csv

ecommerce_orders_dataset.csv

ecommerce_products_dataset.csv

ecommerce_sellers_dataset.csv

product_category_name_translation.csv


In [3]:
import sqlite3 as sql


#Conexion a la base de datos
conn = sql.connect('ecommerce.db')
cursor = conn.cursor()

#script de creacion de tablas
script = '''
CREATE TABLE products (
    product_id TEXT PRIMARY KEY,
    product_category_name TEXT,
    product_name_lenght REAL,
    product_description_lenght REAL,
    product_photos_qty REAL,
    product_weight_g REAL,
    product_length_cm REAL,
    product_height_cm REAL,
    product_width_cm REAL
);
CREATE TABLE order_items (
    order_id TEXT,
    order_item_id INTEGER,
    product_id TEXT,
    seller_id TEXT,
    shipping_limit_date TEXT,
    price REAL,
    freight_value REAL,
    PRIMARY KEY (order_id, order_item_id)
);
CREATE TABLE order_payments (
    order_id TEXT,
    payment_sequential INTEGER,
    payment_type TEXT,
    payment_installments INTEGER,
    payment_value REAL,
    PRIMARY KEY(order_id,payment_sequential)
);
CREATE TABLE orders (
    order_id TEXT PRIMARY KEY,
    customer_id TEXT,
    order_status TEXT,
    order_purchase_timestamp TEXT,
    order_approved_at TEXT,
    order_delivered_carrier_date TEXT,
    order_delivered_customer_date TEXT,
    order_estimated_delivery_date TEXT
);
CREATE TABLE customers (
    customer_id TEXT PRIMARY KEY,
    customer_unique_id TEXT,
    customer_zip_code_prefix INTEGER,
    customer_city TEXT,
    customer_state TEXT
);
CREATE TABLE geolocation(
  geolocation_zip_code_prefix INTEGER PRIMARY KEY,
  geolocation_lat REAL,
  geolocation_lng REAL,
  geolocation_city TEXT,
  geolocation_state TEXT
);
CREATE TABLE order_review(
  review_id TEXT,
  order_id TEXT,
  review_score INT,
  review_comment_title TEXT,
  review_comment_message TEXT,
  review_creation_date TEXT,
  review_answer_timestamp TEXT,
  PRIMARY KEY(order_id,review_id)
);
CREATE TABLE sellers(
  seller_id TEXT PRIMARY KEY,
  seller_zip_code_prefix INTEGER,
  seller_city TEXT,
  seller_state TEXT
);
CREATE TABLE category(
  product_category_name TEXT,
  product_category_name_english TEXT
);
'''
# cursor.executescript(script)
conn.commit()

Inserción de los datos de los csv a la base de datos

In [ ]:
geolocation.to_sql('geolocation',conn, index=True, if_exists='replace')
customers.to_sql('customers',conn, index=True, if_exists='replace')
order_items.to_sql('order_items',conn, index=True, if_exists = 'replace')
order_payments.to_sql('order_payments',conn, index=True, if_exists = 'replace')
order_reviews.to_sql('order_review',conn, index=True, if_exists = 'replace')
orders.to_sql('orders',conn, index=True, if_exists = 'replace')
products.to_sql('products',conn, index=True, if_exists = 'replace')
sellers.to_sql('sellers',conn, index=True, if_exists = 'replace')
category_names.to_sql('category',conn, index=True, if_exists = 'replace')

Creación de tabla geolocation2 con latitudes y longitudes promedio

Con el fin de evitar la duplicidad de datos usando el criterio de promediar latitudes y longitudes agrupando por código postal.

In [ ]:
query_geo = '''
  SELECT geolocation_zip_code_prefix, AVG(geolocation_lat) as promedio_latitud, AVG(geolocation_lng) as promedio_longitud, geolocation_city, geolocation_state
  FROM geolocation
  GROUP BY geolocation_zip_code_prefix
  ORDER BY geolocation_zip_code_prefix;
  '''
geo_2 = pd.read_sql_query(query_geo, conn)
print(geo_2.head(5))

geo_2.set_index('geolocation_zip_code_prefix', inplace=True)

In [ ]:
geo_2.to_sql('geolocation2', conn, index=True, if_exists='replace')

## BBT1-18 - Extracción de datos desde las tablas customers y orders

**Descripción**

COMO: desarrollador

QUIERO: Crear un DataFrame que contenga el JOIN de la tabla customers y tabla orders.

PARA: Extraer los datos de clientes con sus ordenes

In [ ]:
query_1 = '''
SELECT c.*, o.*
FROM customers AS c
JOIN orders as o ON c.customer_id = o.customer_id;

'''
df_customers_orders = pd.read_sql_query(query_1, conn)

df_customers_orders.head(5)

## BBT1-19  Extracción de datos desde las tablas orders y orders_items

**Descripción**

COMO: desarrollador

QUIERO: Crear un DataFrame que contenga el JOIN de la tabla orders y tabla orders_items.

PARA: Extraer los datos de ordenes con sus items

In [ ]:
query_2 = '''
SELECT o.*, oi.*
FROM orders AS o
JOIN order_items AS oi ON o.order_id = oi.order_id;
'''
df_orders_ordersitems = pd.read_sql_query(query_2,conn)
df_orders_ordersitems.head(5)

## BBT1-20  Extracción de datos desde las tablas orders_items y sellers

**Descripción**

COMO: desarrollador

QUIERO: Crear un DataFrame que contenga el JOIN de la tabla orders_items y tabla sellers.

PARA: Extraer los datos de items con por sellers

In [ ]:
query_3 = '''
SELECT oi.*, s.*
FROM order_items AS oi
JOIN sellers AS s ON oi.seller_id = s.seller_id;
'''
df_orderitems_sellers = pd.read_sql_query(query_3, conn)
df_orderitems_sellers.head(5)

## BBT1-21  Extracción de datos desde las tablas orders_items y products

**Descripción**
COMO: desarrollador

QUIERO: Crear un DataFrame que contenga el JOIN de la tabla orders_items y tabla products.

PARA: Extraer los datos de items con por categorias

In [ ]:
BBT1_21_query = '''
SELECT oi.*, p.*
FROM order_items oi
JOIN products p ON oi.product_id = p.product_id
ORDER BY p.product_category_name ASC;
'''

BBT1_21 = pd.read_sql_query(BBT1_21_query, conn)
BBT1_21.head(5)

## BBT1-22 Extracción de datos desde las tablas order y payments

**Descripción**
COMO: desarrollador

QUIERO: Crear un DataFrame que contenga el JOIN de la tabla orders y tabla payments.  

PARA: Extraer los datos de órdenes con sus pagos asociados

In [ ]:
BBT1_22_query = '''
  SELECT o.*, op.*
  FROM orders o
  JOIN order_payments op ON o.order_id = op.order_id;
'''

BBT1_22 = pd.read_sql_query(BBT1_22_query, conn)
BBT1_22.head(5)

## BBT1-23 Extracción de datos desde las tablas customers y geolocation2

**Descripción**
COMO: desarrollador

QUIERO: Crear un DataFrame que contenga el JOIN de la tabla customers y tabla geolocation.
  
PARA: Extraer los datos de customers con sus latitudes y longitudes

In [ ]:
query_bbt1_23 = '''
  SELECT customer_unique_id, geolocation_zip_code_prefix, promedio_latitud, promedio_longitud
  FROM customers c
  JOIN geolocation2 g
  ON g.geolocation_zip_code_prefix = c.customer_zip_code_prefix
  GROUP BY customer_unique_id
  ORDER BY customer_zip_code_prefix
  '''
cust_geo = pd.read_sql_query(query_bbt1_23, conn)
cust_geo.head(5)

## BBT1-24 Extracción de datos desde las tablas orders y geolocation2

**Descripción**
COMO: desarrollador

QUIERO: Crear un DataFrame que contenga el JOIN de la tabla orders y tabla geolocation.
  
PARA: Extraer los datos de órdenes con sus latitudes y longitudes

In [ ]:
query_bbt1_24 = '''
  SELECT order_id, geolocation_zip_code_prefix, promedio_latitud, promedio_longitud
  FROM customers c
  JOIN orders o
  ON c.customer_id = o.customer_id
  JOIN geolocation2 g
  ON g.geolocation_zip_code_prefix = c.customer_zip_code_prefix
  ORDER BY geolocation_zip_code_prefix
  LIMIT 10
  '''
orders_geo = pd.read_sql_query(query_bbt1_24, conn)
orders_geo.head(5)

## BBT1-25 Extracción de datos desde las tablas orders y reviews

**Descripción**

COMO: desarrollador

QUIERO: Crear un DataFrame que contenga el JOIN de la tabla orders y tabla reviews.

PARA: Extraer los datos de órdenes con sus scores

In [ ]:
query_bbt1_25 = '''
  SELECT o.order_id, review_score
  FROM orders o
  JOIN order_review r
  ON o.order_id = r.order_id
  ORDER BY review_score
  LIMIT 10
  '''
orders_score = pd.read_sql_query(query_bbt1_25, conn)
orders_score.head(5)

## BT1-26 Extracción de datos desde las tablas orders_items, products y products_category_translation

**Descripción**

COMO: desarrollador

QUIERO: Crear un DataFrame que contenga el JOIN de la tabla orders_items, tabla products y product_category_name_translation.  

PARA: Extraer los datos de items con por categorias pero con su nombre traducido en inglés

In [ ]:
bt1_26_query = '''
    SELECT oi.*, c.product_category_name_english AS product_name
    FROM order_items AS oi
    INNER JOIN products AS p
        ON oi.product_id = p.product_id
    INNER JOIN category AS c
        ON p.product_category_name = c.product_category_name_english
;
'''
BT1_26 = pd.read_sql_query(bt1_26_query, conn)
BT1_26.head(5)

## BT1_27  Extracción de datos desde las tablas orders, payments y customers

**Descripción**

COMO: desarrollador

QUIERO: Crear un DataFrame que contenga el JOIN de la tabla orders, tabla payments y customers.
  
PARA: Extraer los datos de clientes con sus métodos de pagos por ciudad.

In [ ]:
bt1_27_query = '''
    SELECT c.customer_unique_id, op.payment_type, c.customer_city, c.customer_state
    FROM customers AS c
    INNER JOIN orders AS o
        ON c.customer_id = o.customer_id
    INNER JOIN order_payments AS op
        ON o.order_id = op.order_id

;
'''
BT1_27 = pd.read_sql_query(bt1_27_query, conn)

BT1_27.groupby(['payment_type'])[['customer_unique_id']].count().sort_values(by='customer_unique_id', ascending=False)

BT1_27.head(5)

## BBT1-35 - Filtrar datos, reducir volumen de orders y reviews

In [17]:
query_bbt1_35 = '''
    SELECT
        o.order_id,
        o.order_item_id,
        o.product_id,
        o.seller_id,
        o.shipping_limit_date,
        SUM(o.price) AS total_price,
        o.freight_value        
    FROM order_items AS o
    INNER JOIN order_review AS r
        ON o.order_id = r.order_id
    WHERE r.review_score >= 3
    GROUP BY o.order_id
;
'''
# bt1_35 = pd.read_sql_query(query_bt1_35, conn).reset_index(drop=True)
bbt1_35 = pd.read_sql_query(query_bbt1_35, conn)
bbt1_35.head(5)

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,total_price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


## BBT1-36 - Filtrar datos, reducir volumen de orders_items, products y products_category_translation

- COMO: desarrollador
- QUIERO: Filtrar el DataFrame de las órdenes con mayor o igual importe al promedio de venta y resguardarlos junto con la información de la categoría traducida.   
- PARA: Reducir el volumen de datos y trabajar solo con las órdenes de mayor ingreso al promedio y entender cuales son las categorías de los productos..

In [21]:
query_bbt1_36 = '''
    SELECT oi.order_id, c.product_category_name_english, oi.price
    FROM order_items AS oi
    INNER JOIN products AS p
        ON oi.product_id = p.product_id
    INNER JOIN category AS c
        ON p.product_category_name = c.product_category_name
    WHERE oi.price > (SELECT avg(price) FROM order_items)
    GROUP BY oi.order_id
    ORDER BY oi.price DESC
;
'''
bbt1_36 = pd.read_sql_query(query_bbt1_36, conn)
bbt1_36.head(5)

,order_id,product_category_name_english,price
0,0812eb902a67711a1cb742b3cdaa65ae,housewares,6735.0
1,fefacc66af859508bf1a7934eab1e97f,computers,6729.0
2,f5136e38d1a14a4dbd87dff67da82701,art,6499.0
3,a96610ab360d42a2e5335a3998b4718a,small_appliances,4799.0
4,199af31afc78c699f0dbf71fb178d4d4,small_appliances,4690.0


Descripción

- COMO: desarrollador
- QUIERO: Filtrar el DataFrame de órdenes para resguardar las revisiones con puntaje mayor o igual a 3 y mayor importe de la órden.   
- PARA: Reducir el volumen de datos y trabajar solo con las revisiones de las órdenes con mayor ingreso.

## BBT1-37 - Filtrar datos, reducir volumen de orders, payments y customers

Descripción

COMO: desarrollador
QUIERO: Filtrar el DataFrame de órdenes tomando sólo aquellos compras realizadas tarjeta de crédito y cuyo importe sea mayor que el promedio de los pagos resguardando la información requerida junto con las ciudades de los clientes.   
PARA: Reducir el volumen de datos y trabajar solo con aquelas órdenes que tienen un monto representativo con tarjeta de crédito y entender de que ciudades corresponden.

In [23]:
query_bbt1_37 = '''
    SELECT
        o.*,
        op.payment_type,
        c.customer_city,
        c.customer_state
    FROM orders as o
    INNER JOIN order_payments AS op
        ON o.order_id = op.order_id
    INNER JOIN customers AS c
        ON o.customer_id = c.customer_id
    WHERE op.payment_type = 'credit_card'
        AND op.payment_value > (SELECT avg(payment_value) FROM order_payments)
;
'''
bbt1_37 = pd.read_sql_query(query_bbt1_37, conn)
bbt1_37.head(5)

,index,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,payment_type,customer_city,customer_state
0,90161,1f78449c87a54faf9e96e88ba1491fa9,68c9dafdf193586c290b19c71541357a,delivered,2018-07-22 14:07:35,2018-07-23 11:32:08,2018-07-23 12:58:00,2018-07-28 15:03:04,2018-08-14 00:00:00,credit_card,campina grande,PB
1,42220,d88e0d5fa41661ce03cf6cf336527646,fe0c5425832ac53c3be5cb4aea25b60f,delivered,2017-11-28 15:53:29,2017-11-28 16:17:30,2017-12-12 23:22:02,2017-12-19 20:49:52,2017-12-18 00:00:00,credit_card,marilia,SP
2,16686,12e5cfe0e4716b59afb0e0f4a3bd6570,041cd6848abb3d3ca45e83dc971488fd,delivered,2018-03-22 09:46:07,2018-03-23 09:48:20,2018-03-28 15:32:48,2018-04-26 16:38:53,2018-04-12 00:00:00,credit_card,cachoeiro de itapemirim,ES
3,56184,8ac09207f415d55acff302df7d6a895c,2d8bf5f6862af587af2e0b085a04bf0d,delivered,2018-01-29 14:50:01,2018-01-29 15:18:37,2018-01-30 17:36:57,2018-02-01 15:43:08,2018-02-14 00:00:00,credit_card,diadema,SP
4,76165,4214cda550ece8ee66441f459dc33a8c,be4ecdb7c9401fd98ed01456634cfcb2,delivered,2017-03-21 17:39:19,2017-03-21 17:39:19,2017-03-22 10:26:53,2017-03-30 15:05:20,2017-04-25 00:00:00,credit_card,pacaja,PA


## BBT1-45 - Carga de datos, generacion de archivos .csv de orders y reviews

Descripción

- COMO: desarrollador
- QUIERO: Generar archivos .csv a partir del Dataframe generado de orders y reviews. Los archivos deben cumplir los siguientes requerimientos: 
   - Contener cabecera. 
   - Separador “,”. 
   - Ser guardados en un directorio llamado Results\Analysis_H.
- PARA: Resguardar los datos enriquecidos, según requerimiento.

In [ ]:
'''
Formato para hacer las consultas

BT1_NN.to_sql('Query_BT1_NN', conn, index=True, if_exists='replace')
'''

In [20]:
#Nombre del archivo.csv
csv_filename = 'BBT1-35.csv'

# Ruta completa del archivo CSV
csv_path = 'Results/Analysis_H/' + csv_filename

bbt1_35.to_csv(path_or_buf=csv_path, index=False, sep=',', header=True)

print(f'Archivo CSV guardado en: {csv_path}')

Archivo CSV guardado en: Results/Analysis_H/BBT1-35.csv


## BBT1-46 - Carga de datos, generacion de archivos .csv de orders_items, products y products_category_translation

Descripción

- COMO: desarrollador
- QUIERO: Generar archivos .csv a partir del Dataframe generado de orders_items, products y products_category_translation. Los archivos deben cumplir los siguientes requerimientos: 
    - Contener cabecera. 
    - Separador “,”. 
    - Ser guardados en un directorio llamado Results\Analysis_I.
PARA: Resguardar los datos enriquecidos, según requerimiento.

In [25]:
#Nombre del archivo.csv
csv_filename = 'BBT1-36.csv'

# Ruta completa del archivo CSV
csv_path = 'Results/Analysis_I/' + csv_filename

bbt1_36.to_csv(path_or_buf=csv_path, index=False, sep=',', header=True)

print(f'Archivo CSV guardado en: {csv_path}')

Archivo CSV guardado en: Results/Analysis_I/BBT1-36.csv


## BBT1-47 - Carga de datos, generacion de archivos .csv de orders, payments y customers

Descripción

- COMO: desarrollador
- QUIERO: Generar archivos .csv a partir del Dataframe generado de orders, payments y customers. Los archivos deben cumplir los siguientes requerimientos: 
    - Contener cabecera. 
    - Separador “,”. 
    - Ser guardados en un directorio llamado Results\Analysis_J.
- PARA: Resguardar los datos enriquecidos, según requerimiento.

In [26]:
#Nombre del archivo.csv
csv_filename = 'BBT1-37.csv'

# Ruta completa del archivo CSV
csv_path = 'Results/Analysis_J/' + csv_filename

bbt1_37.to_csv(path_or_buf=csv_path, index=False, sep=',', header=True)

print(f'Archivo CSV guardado en: {csv_path}')

Archivo CSV guardado en: Results/Analysis_J/BBT1-37.csv


## BBT1-55 - Extraccion de datos desde .csv, análisis H

Descripción

- COMO: desarrollador
- QUIERO: Cargar los datos del análisis H generados en un DataFrame.
- PARA: Poder analizar el conjunto de datos.

In [7]:
# !pip install pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [8]:
path_bbt1_35 = './Results/Analysis_H/BBT1-35.csv'
bbt1_35_spark = spark.read.load(path_bbt1_35, format='csv', sep=',', inferSchema='true', header='true')
bbt1_35_spark.show(5)

+--------------------+-------------+--------------------+--------------------+-------------------+-----------+-------------+
|            order_id|order_item_id|          product_id|           seller_id|shipping_limit_date|total_price|freight_value|
+--------------------+-------------+--------------------+--------------------+-------------------+-----------+-------------+
|00010242fe8c5a6d1...|            1|4244733e06e7ecb49...|48436dade18ac8b2b...|2017-09-19 09:45:35|       58.9|        13.29|
|00018f77f2f0320c5...|            1|e5f2d52b802189ee6...|dd7ddc04e1b6c2c61...|2017-05-03 11:05:13|      239.9|        19.93|
|000229ec398224ef6...|            1|c777355d18b72b67a...|5b51032eddd242adc...|2018-01-18 14:48:30|      199.0|        17.87|
|00024acbcdf0a6daa...|            1|7634da152a4610f15...|9d7a1d34a50524090...|2018-08-15 10:10:18|      12.99|        12.79|
|00042b26cf59d7ce6...|            1|ac6c3623068f30de0...|df560393f3a51e745...|2017-02-13 13:57:51|      199.9|        18.14|


## BBT1-56 - Extraccion de datos desde .csv, análisis I

Descripción

- COMO: desarrollador
- QUIERO: Cargar los datos del análisis I generados en un DataFrame.
- PARA: Poder analizar el conjunto de datos.

In [9]:
path_bbt1_36 = './Results/Analysis_I/BBT1-36.csv'
bbt1_36_spark = spark.read.load(path_bbt1_36, format='csv', sep=',', inferSchema='true', header='true')
bbt1_36_spark.show(5)

+--------------------+-----------------------------+------+
|            order_id|product_category_name_english| price|
+--------------------+-----------------------------+------+
|0812eb902a67711a1...|                   housewares|6735.0|
|fefacc66af859508b...|                    computers|6729.0|
|f5136e38d1a14a4db...|                          art|6499.0|
|a96610ab360d42a2e...|             small_appliances|4799.0|
|199af31afc78c699f...|             small_appliances|4690.0|
+--------------------+-----------------------------+------+
only showing top 5 rows



## BBT1-57 - Extraccion de datos desde .csv, análisis J

Descripción

- COMO: desarrollador
- QUIERO: Cargar los datos del análisis J generados en un DataFrame.
- PARA: Poder analizar el conjunto de datos.

In [10]:
path_bbt1_37 = './Results/Analysis_J/BBT1-37.csv'
bbt1_37_spark = spark.read.load(path_bbt1_37, format='csv', sep=',', inferSchema='true', header='true')
bbt1_37_spark.show(5)

+-----+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+------------+--------------------+--------------+
|index|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|payment_type|       customer_city|customer_state|
+-----+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+------------+--------------------+--------------+
|90161|1f78449c87a54faf9...|68c9dafdf193586c2...|   delivered|     2018-07-22 14:07:35|2018-07-23 11:32:08|         2018-07-23 12:58:00|          2018-07-28 15:03:04|          2018-08-14 00:00:00| credit_card|      campina grande|            PB|
|42220|d88e0d5fa